# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [12]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [9]:
# https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
def scroll_down(self):
    """A method for scrolling the page."""

    # Get scroll height.
    last_height = self.driver.execute_script("return document.body.scrollHeight")

    while True:

        # Scroll down to the bottom.
        self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load the page.
        time.sleep(2)

        # Calculate new scroll height and compare with last scroll height.
        new_height = self.driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:

            break

        last_height = new_height

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [13]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [ ]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#

while True:
    # Scroll down to the bottom.
    last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(2)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:

        break

    last_height = new_height

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [15]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [16]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 影／小資女10大存錢法大公開　奈米資金拚存百萬青春不浪費

記者吳靜君／台北報導
有許多小資女、年輕的女生常嘆「賺錢這麼少，用都不夠，如何存錢？」、「存錢好難喔」。專家教你聰明存錢十大招，即使是小資女、奈米投資也可以輕鬆存100萬元、第一桶金，並且儲蓄時間大大縮短，青春不用浪費。
▲小資女即使收入不多、可運用的資金有限，但是只要掌握十招，就可以短時間達到理財目標，青春不浪費。（圖／取自免費圖庫、示意圖）
第一招：訂立目標　累積儲蓄的動力
台灣財富管理顧問公司董事執行長陳宥綸表示，理財的第一步就是要訂立目標，像是購屋、結婚還是出國旅遊等，有了目標，就會有動力存錢，不會讓金錢浪費在「不知不覺中」。
第二招：計算達成目標的所需時間
訂立目標之後，就要確定目標要多少錢、要花多少時間的儲蓄，像是如果要存第一桶金100萬元，若是每個月存1萬元，若沒有任何投資，就必須要花8年多才會存到，所以要縮短時間就是要增加每個月的儲蓄或者是適當的投資，縮短達到目標的時間，陳宥綸建議，小資女可以運用坊間有許多計算工具、應用程式（App）都可以幫助她們計算每月要存多少錢、投資報酬率要達多少、多久才能達成目標。
第三招：設定預算　擠出儲蓄理財資金
一般小資女的薪水大約3萬多到4萬以上，但不論薪水高低，小資女可以用預算法來支配生活的開支，例如每個月固定開支可以設定預算，伙食費6,000元、娛樂費2,000元、房租6,000元等、生活費例如交通費、學貸等，薪水扣除這些預算之後，或多或少都可以擠出3,000～5,000元以上，養成強迫儲蓄、強迫定額投資，是累積第一桶金的重要步驟。
第四招：年終獎金、紅包等額外收入不亂花
隨著年末來臨，公司、長輩如果有給年終獎金、紅包等，除了必要的支出，像是孝敬長輩或疼愛晚輩紅包等花費，盡量控制在1/3之內，剩餘的2/3甚至更高應該保留下來，作為儲蓄或者是緊急預備金之用。
第五招：儲存緊急預備金　意外來了不擔心
單身貴族、小資女都要有緊急預備金的概念，一旦要追求更好的工作或者是遇到意外狀況時，就不會不知所措，而這筆緊急預備金是用來應付緊急狀況、風險之用，建議緊急預備金安全水位起碼要應付3個月的生活必須花費，也就是即使3個月

▲ 投票日上班雇主應給勞工加班費。（圖／pakutaso）
記者李瑞瑾／報導
勞動部已將明年總統大選投票日訂為勞基法規定的放假日，雇主應依法讓勞工放假一日，但如果當天仍有出勤的勞工，則依法可以拿到加班費。
勞動部表示，為保障具投票權的勞工得以行使參政權，勞動部已指定選舉投票日為「勞動基準法」所定放假日，因此，109年1月11日（星期六）2020總統大選投票當日，具有投票權且原屬工作日的勞工，雇主應依法讓勞工放假一日，工資照給。至於具投票權但當日原屬勞工的休息日或例假者，因勞工本得於當日行使投票權，因此不另外給假。
勞動部表示，放假「一日」指的是午前零時至午後12時連續24小時。由於投票權僅得於投票當日行使，性質與一般國定假日不同，因此，投票日不得再與其他工作日對調實施。
勞動部進一步說明，雇主如果徵得具投票權勞工同意於投票日出勤工作，應不妨礙其投票，並應加給當日出勤工資，至於計算方式分為原為上班日，與原為休息日兩種類型。
▲ 兩種情況讓加班費計算方式不同。（圖／pixabay）
一、 原屬勞工的工作日：應加倍發給出勤時段的工資。舉例而言：勞雇雙方約定當天出勤8小時，勞工外出投票2小時，返回工作6小時，雇主須加給6小時的工資。
二、 原屬勞工的休息日：視工作時數依「勞動基準法」第24條第2項有關休息日出勤加給之規定計算。
最後，勞動部再次提醒，雇主未依法給假或給薪者，可處新臺幣2萬元以上，最高100萬元罰鍰，並應補給工資。勞工權益如有受損害者，可就近向工作所在地勞工行政主管機關（即當地直轄市、縣市政府之勞工或社會局、處）申訴，以維權益。
----------------------------------------------------------------------

[8] 展望2020年　東森社交電商啟動全球佈局展開5年計畫

▲東森社交電商高峰會 王令麟霸氣喊「成為台灣直銷第一名」。(採訪撰稿／記者黃稜涵；攝影剪輯／記者謝盛帆)
▲東森全球新連鎖事業總經理彭振東表示，社交平台網連通準備於2020年進軍大陸、東南亞，5年目標營收做到690億台幣。（圖／東森全球新連鎖事業提供）
記者洪菱鞠／台中報導
第三屆東森社交電商大會28日在台中盛大舉辦，今年以「創新求變·攜手共創新電商時代」為主題，除回顧過去近一年來經營成果，也分享新型美麗站長計畫以及串聯層面更廣的

▲ 年終獎金算是上班族辛苦一年的大紅包！（圖／資料照）
記者李瑞瑾／台北報導
中美貿易戰雖造成景氣動盪，但也讓台廠成為了其中的受惠者，尤其傳產像是水泥、紡織業者，今年獲利可期，其中紡織股王儒鴻今年的年終獎金可望比照去年水準，最高上看10個月，讓不少上班族大感羨慕！
其他人也看了這些～～
► 年終獎金大公開！金融業九連霸領先　兆豐發8個月、中油4.4、住商3個月讓人羨
----------------------------------------------------------------------

[15] 新一代iPad Pro外觀曝光！搭載三顆鏡頭　預計明年3月推出

▲ 2020 iPad Pro。（圖／翻攝自YouTube／iGeeksBlog）
記者李瑞瑾／綜合報導
爆料大神OnLeaks在最近洩漏了兩款2020 iPad Pro的外觀渲染圖與360°影片，圖片顯示新款iPad Pro將追隨iPhone 11 Pro的腳步，搭載三鏡頭模組。
國外科技媒體iGeeksBLOG報導指出，他們收到了OnLeaks洩漏的兩款iPad Pro渲染圖和影片，兩款尺寸分別為11吋和12.9吋。OnLeaks表示，11吋的iPad Pro尺寸可能約為248 x 178.6 x 5.9mm，厚度中也包括後置鏡頭凸起部分的7.8mm。
除了高度和寬度之外，OnLeaks指出，iPad Pro的厚度可能會比前幾代更多一些，因為蘋果計劃為新一代iPad Pro安裝三顆鏡頭的相機模組。
OnLeaks表示，無法確認兩種型號的機身會使用哪種材料（鋁或玻璃），為了模擬這兩種可能性，11吋的渲染模型採用了普通的鋁製機身，而12.9吋的模型採用了玻璃背蓋。
OnLeaks預期，蘋果很可能會在2020年春季發布這兩款iPad Pro，極有可能在明年3月推出；此外，考慮到蘋果一直以來的作業模式，蘋果將在這款iPad Pro中安裝A13X晶片。
----------------------------------------------------------------------

[16] 外媒回顧近十年14支最重要安卓手機　三星走在前端、佔掉近一半席次！

▲ 外媒回顧10年來安卓手機發展，共整理出14支最重要的安卓陣營智慧型手機。（圖／達志影像／美聯社）
記者李瑞瑾／綜合報導

記者姚惠茹／台北報導
軒郁國際（6703）興櫃滿一年多，創辦人暨總經理楊尚軒回首創立十年來的打拚過程，從一片面膜都賣不出去的窘境，如今每年可以賣出一億片面膜，靠得就是打破傳統面膜的「耳掛面膜」、「黑面膜」，出奇制勝的打造出屬於自己的面膜王國。
楊尚軒回憶，當年因為太太想要找尋自己用的保養品，說到「每個女生都因為面膜讓生活變得更簡單」，就因為老婆的一句話，創立「軒郁國際」踏入面膜美妝產業，並以「SEXYLOOK」這個品牌名稱在市場打出知名度。
▲軒郁國際創辦人暨總經理楊尚軒。（圖／記者姚惠茹攝）
楊尚軒表示，其實創業之初並不容易，當時市面上已經有好多面膜品牌，因此為了做出市場區隔，殺出一條血路，只能「出奇制勝」推出第一款商品「耳掛式雙拉提面膜」，並透過電視購物的教育推廣模式，最高曾經一分鐘賣出2千片面膜，可以說是掀起一股面膜風潮。
楊尚軒指出，不久就觀察到消費者對於保養品牌會有忠誠度，但對於面膜卻有著喜歡嚐鮮的心理，尤其是台灣女生喜歡到髮廊洗頭，洗完之後就不想要戴上耳掛式面膜，而且並非每個人都喜歡如何緊貼的面膜，因此決心開發下一款面膜商品。
▲軒郁國際創辦人暨總經理楊尚軒做出市場第一片的「黑面膜」。（圖／記者呂佳賢攝）
楊尚軒進一步指出，第二款商品為了打破傳統對面膜只有白色的印象，因此就做出市場第一片的「黑面膜」，卻因為黑的真的太怪，面臨一片面膜都賣不出去的窘境，直到陸綜有女藝人戴上黑面膜玩遊戲，終於讓這款面膜在一夜爆紅，甚至達到供不應求的情況。
楊尚軒提到隨後軒郁開始專走「不一樣的面膜」，目前旗下共有三個面膜品牌，包括SEXYLOOK極美肌、MIRAE未來美、LUDEYA露蒂雅、Dr.May美博士，並陸續推出只要敷8分鐘的快速面膜，以及創新研發的酵素面膜，而且至今仍不斷朝向創新的道路前進。
▲軒郁旗下品牌MIRAE未來美推出的8分鐘面膜。（圖／記者呂佳賢攝）
----------------------------------------------------------------------

[21] 影／2020年轉機股、地雷股　資深分析師一次告訴你

記者陳心怡／台北報導
近年來，「存股族」越來越流行，但目前台股已經站上1萬2000點之上，有哪些還適合存股呢？在《雲端最有錢》節目中，「飆股達人」朱家泓就提到，金融股、台塑四寶和航運股適合。
朱家泓提到，目

文╱謝明彧 　攝影╱關立衡
《遠見雜誌》從2016年起設立華人社會首創的「君子企業家獎」，表彰對人類發展帶來正向影響的企業家。
今年則將表彰的人物從企業家擴大到科學家，首度頒發「君子科學家獎」，得獎的是中央研究院院士暨美國國家科學院院士錢煦。
錢煦曾獲中華民國總統生命科學獎獎章及美國國家科學獎獎章，是唯一榮獲兩國最高科學獎章的科學家；並身兼美國四大科學院（國家醫學科學院、國家工程科學院、國家科學院、藝術及科學學院）及中國科學院外籍院士，是當今華人社會唯一有此榮銜的科學家。
頒獎會場上特別邀請國泰慈善基金會董事長，也是錢煦親弟弟的錢復擔任引言人，並由錢煦母校台灣大學前校長、台大名譽教授孫震擔任贈獎貴賓。以下為贈獎典禮與演講精華：
《四書》中有一句話：「君子慎其獨也」，意思是君子私底下是非常謹慎，不會做任何不好的事。
我二哥錢煦的工作與研究非常忙碌，偶爾有時間回到家，吃過晚飯後不是看書、看研究期刊，就是寫文章、做論文。夫人總是不忘記說一句話：「該休息了。」二哥的答覆永遠都是：「這一段寫完就去休息，」但還是繼續忙碌，直到深夜。
聽到這件事，我每每都想，「如此『慎獨』的人，怎麼可能不是君子呢？」《論語》「魏靈公章」還有一句話，「君子群而不黨」。二哥1969年在美國哥倫比亞大學有自己的研究室，同仁20多位，1988年他到加州大學聖地牙哥校區任教，即使有這麼多朋友，但他從來不營私結黨。
1972年，二哥被提名參選中研院院士，那時我父親是中研院院長，父親為了避嫌，連續兩回拜託院士，千萬不要讓錢煦當上院士。直到四年後，我父親沒有主持遴選會議，二哥才以最高票當選。
「君子」是孔子所塑造的理想人格典範，也是儒家核心倫理具體的呈現。君子懷抱著仁愛的關懷世人之心，己欲利而利人，己欲達而達人，是中華文化中最美好的形象。
現在社會最需要的，就是君子的人格，如果企業家是君子，企業就不會對社會和環境帶來傷害。如果政治領袖是君子，今天的世界不會這麼紛擾和衝突，台灣老百姓也能幸福過日子。
我對君子企業家、君子科學家都有很高期待，也期待有一天可以看到「君子政治領袖」出現，讓邪惡政客改邪歸正，讓孔子、周公嚮往的文武周公之治，可以重現。
我自己是做科學的，任何人做任何事業，都離不開人生。所以這次以「科學與人生」做為演講主題。1931年，我在北平（現北京）出生，小學前在北平生活，但隨著蘆溝橋事變爆發，舉家遷到

▲ 中央銀行。（圖／記者李毓康攝）
記者李瑞瑾／台北報導
央行今（27）日公布本國銀行9月國家風險統計，9月底本國銀行外國債權直接風險餘額為4,342億美元，較108年6月底減少11億美元或0.26%，主要是對銀行部門債權減少所致；由於部分直接債務人與其最終債務人或保證人分屬不同國家，經風險移轉後，9月底最終風險淨額為4,172億美元，較108年6月底增加24億美元或0.57%。
外國債權直接風險依部門別分析，以非銀行之私人部門占62.4%為主，銀行占29.79%次之，公共部門則占7.8%；依國家類別分析，以已開發國家占52.5%為主，開發中國家及境外中心分別占24.67%及22.28%；依地區別分析，以亞洲及太平洋地區占47.5%為主，其次為美洲及加勒比海地區占28.86%，歐洲地區占20.42%。
另外，本國銀行6月底外國債權直接風險前十大國家（地區），依序為美國、中國大陸、盧森堡、香港、日本、澳大利亞、英國、開曼群島、新加坡及英屬西印度群島，合計3,197億美元，占外國債權直接風險餘額之73.64%。
其中，9月底對中國大陸直接風險為438億美元，較6月底減少14億美元（或3.01%）；最終風險為651億美元，較6月底減少28億美元(或4.13%)
央行官員表示，美國債權直接風險因資金拆存及證券化商品投資增加，續創歷史新高紀錄，且已連續17季位居外國債權直接風險第一大國；而對中國大陸直接風險減少，除了受金融債券投資跟資金拆存減少這兩項因素之外，也因人民幣貶值，致使換算成美元統計之後的債權金額減少。
----------------------------------------------------------------------

[39] 爭取5G入場不手軟！競標14天達746.42億　30日挑戰800億

----------------------------------------------------------------------

[40] 歷經3屆奧運挫敗...國巨陳泰銘二公主陳少曼取得2020東奧馬術門票　

▲國巨二公主陳少曼取得東奧馬術門票。（圖／陳少曼提供）
記者周康玉／台北報導
國巨（2327）董事長陳泰銘次女陳少曼於日前（12/21）在沙烏地阿拉伯舉辦的奧運個人積分賽中，已取得絕對領先，陳少曼今（27）日對外表示，已經

年終獎金與經濟成長率呼應，2014~2019年經濟成長率高峰落在2014年的4.72%，當年企業年終平均1.31個月，也是這六年最高。
人力銀行表示，2019年金融業平均年終1.64個月奪產業之冠，十年連九霸。
金融
至於民營的國泰金，要看公司營運績效、盈餘、員工績效表現核發年終獎金；中信金也說將比照往年，依獲利水準提撥年終獎金，預計農曆年前發放。
手搖飲料連鎖店
手搖飲市場競爭激烈，全台約有2萬2000多家手搖品，其中，台灣在地品牌包括發跡台北士林的「珍煮丹」以及台南佳里的「迷客夏」依舊穩健成長中，珍煮丹在台擁有54間門市，今年營業額相較去年成長2成5，第4季又比第3季成長3成5。
房產
信義房屋及永慶房產集團兩大房仲龍頭，都尚未公布年終獎金，至於台灣房屋也尚未公布，預計要到過年前，才會有明確訊息。
傳產
台塑集團表示，年終獎金等一月份的業績說明會才會有確定的數字，目前媒體所說的皆為自行推測。
科技
鴻海（2317）每年都會在年底發放員工分紅，今年提早到24日發放入帳，總金額達76.62億元，預計以公開資訊數據顯示的員工人數4,166人估算，平均每位員工可領到約183萬元，但個別員工實際發放情況，仍須依據單位績效與職等而有所不同，甚至未獲利單位員工有可能掛零。
人力銀行
其他人也看了這些～～
► 超羨慕！傳產「年終王」獎金最高上看10個月　最低也有3個月
----------------------------------------------------------------------

[48] 權值股領漲！　台股收盤上漲90點收12,091點

▲ 台股。（資料照／記者湯興漢攝）
記者李瑞瑾／台北報導
受到美股續創新高激勵，權值股領大盤上衝，讓台股今（27）天開高走高，收盤上漲90點或0.75%，收在12,091點，成交量1,083.79億元。
電子三雄中，台積電（2330）開高走高，收盤在338元、上漲1.5%，鴻海（2317）收在91.5元、上漲0.77%；股王大立光（3008）早盤一度順利站回5千元大關，但隨後又收斂到4990元附近震盪，終場收在4995元、上漲1.11%。
台塑四寶也全數收高，台塑化（6505）、台塑（1301）漲逾1%，漲幅分別為1.75%與1.2%，南亞（1303）漲0.55%、台化（1326）收漲0.45%。
類股指數漲多

記者陳心怡／新北報導
國民黨總統候選人韓國瑜今（27）日上午偕同副總統候選人張善政，前往宏達電（HTC）參訪。韓國瑜在聯訪時表示，蔡英文總統在政見發表會時「說國民黨貪汙，所以代表民進黨清廉，大錯特錯。」
▲韓國瑜抨擊蔡英文對陳水扁貪污事件完全不敢開口。（圖／記者李毓康攝）
蔡英文在第二場電視政見發表會提到，韓國瑜狂轟蔡政府「貪汙」，不過卻一個具體案例都說不出來，她問：「還記得林益世、李朝卿、許志堅、葉世文嗎？」對此，韓國瑜表示，日前在電視辯論，蔡英文提出幾位國民黨因為貪汙進到監獄裡的案例，來表達對國民黨的不屑和不認同，「我是不想大家淪為口水，因為政見發表會難得有這個機會跟全國民眾講自己關心的以及全國民眾需要的，所以我沒有提出反駁。」
「蔡英文總統一個手指指著我，三個指頭指著自己，只是她自己不知道而已」，韓國瑜認為，蔡英文說國民黨貪汙，所以代表民進黨清廉，「大錯特錯」。
----------------------------------------------------------------------

[60] 影／韓國瑜痛批反滲透法讓全民惶恐！「連媽祖的宗教交流，都可能入罪，叫台灣人情何以堪」

記者李錦奇／綜合報導
韓國瑜今（27）日被問到對反滲透法的看法時，直批是蔡英文與民進黨在用危機意識，催發選票，已經引起全國民眾，與廣大台商甚至台生的惶恐不安。法案一旦通過，「影響所及，將來連媽祖、保生大帝這類的宗教交流，都可能入罪，叫台灣人情何以堪。」
▲韓國瑜與張善政參訪宏達電，王雪紅與陳文琦夫妻親自接待。（圖／記者李毓康攝）
韓國瑜說，反滲透法，將來如何認定，會不會變成羅織入罪，讓人恐懼不已，所以馬英九總統說這是戒嚴的恢復。他也再度呼籲民進黨，此刻強渡反滲透法，還在推銷亡國感，還在用危機意識，來催發自己的選票嗎？還在製造對立嗎？「有必要在距離投票日14天左右，通過爭議性這麼高的反滲透嗎？」
韓國瑜說「影響所及，將來連媽祖、關公、保生大帝，連這類的宗教交流，都可能入罪，叫台灣人情何以堪。」，他還批民進黨藉由政治鬥爭，政治肅殺，製造亡國感，全國人民一定會唾棄。
----------------------------------------------------------------------

[61] 影／遭謝長廷諷「說草率草率到」　韓國瑜：他自己要檢討

財經中心／新北報導
國民黨總統候選人韓國瑜今（27）日上午偕同副總統候選人張善政，前往宏達電（HTC）參訪，董事長王雪紅及董事陳文琦夫婦也親自接待，預計安排韓張兩人體驗《蒙娜麗莎–越界視野》與席科特手術的兩大VR虛擬實境內容。《ETtoday新聞雲》也直播這場參訪活動，提供現場最新直擊畫面。
韓國瑜、張善政這次參訪宏達電位於新北市新店總部，由董事長王雪紅及董事陳文琦親自接待，除了解VR、AR（擴增實境）、AI（人工智慧）、5G及區塊鏈最新發展，並預計體驗宏達電與法國羅浮宮攜手合作的《蒙娜麗莎–越界視野》，以及今年取得TFDA認證的席科特手術導航軟體與席科特手術計畫軟體。
值得一提的是，韓國瑜一行人，除了先在一樓大廳親身體驗VR智慧醫療應用，欣賞7分鐘的VR藝術《蒙娜麗莎–越界視野》，還將和宏達電同仁進行半小時的《青年國是論壇》，暢談「創新未來」，屆時韓張兩人會跟宏達電員工激盪出什麼對話火花，值得拭目以待。
----------------------------------------------------------------------

[74] 中壢市心房市MVP　2字頭人氣公園學區宅

▲「京耀富達」打出2字頭房價，房價親民。
圖、文／京耀富達提供
「京耀富達」位於多才多藝復旦國小文教區，學校歷史悠久、口碑優良，接送孩子上學方便，不用學孟母三遷，又鄰近0.69公頃的廣達公園，有晴雨皆宜的天幕球場，可容納1,500人，擁有購屋首要考量的學區、公園雙優勢，是市區內兼具學區與綠意的稀有優質環境，因此成為許多在地年輕人的購屋首選。
京耀建築．富桃建設深根在地，由專業建築師領導的建築職人團隊，在寸土寸金的市心版圖聯手打造「京耀富達」，一座文青專屬靜巷美宅，讓建築，不只是安心的居所，更是精緻藝術品與追求細緻之美的延伸。
----------------------------------------------------------------------

[75] 新台幣早盤升值1.21角！　最高一度來到30.08元兌1美元

▲ 新台幣兌美元今天開盤貶值0.9分。（圖／路透社）
記者李瑞瑾／台北報導
台北外匯市場新台幣兌美元匯率今（27）天開盤貶值0.9分、以30.21兌1美元開出，開盤之後轉為走揚，並升值逾1角，最高來到30.08元兌1美元。
新台幣兌美

▲玉山銀行今天宣布，明年起全面調薪，其中基層員工加薪幅度達到10%以上。（圖／記者紀佳妘攝）
記者紀佳妘／台北報導
玉山銀行今（26）日宣布，自2020年1月1日起全面調薪，全年平均調薪幅度超過8%，其中基層同仁更達10%以上，大手筆的加薪更羨煞不少上班族。
在長期獲利成長的支持下，玉山銀行為了感謝全體主管同仁的付出與努力，以及鼓勵同仁持續深耕專業、學習成長，共創更好的綜合績效，決定從2020年1月1日起全面調薪，全年平均調薪幅度超過8%。
玉山銀行指出，在1月1日的第一波調薪中，主要聚焦基層同仁及中階主管的薪資結構，提高固定薪的金額，基層同仁每個月本薪提高3,000元，中階主管提高主管加給，全體同仁平均提升4.6%，基層同仁平均提升6.4%。
另在4月1日後，加計第1季經理人、中階主管及同仁的晉升及績效的第二波調薪後，全體同仁平均薪資將提升超過8%，基層同仁平均薪資將提升達10%以上。
玉山銀行認為，人才是企業及國家競爭力的先行指標，也是關鍵指標，除了有形的薪資外，也將提供完整的教育訓練、多元的職涯發展、優質的工作環境、良好的員工福利以及寬廣的發展舞台，讓員工都可以一起為理想而奮鬥，成就精彩豐富的人生。
----------------------------------------------------------------------

[87] 金融研訓院推金融界的「基測」　求職公股銀「一試」搞定

▲金融研訓院推金融基測。（圖／記者吳靜君攝）
記者吳靜君／台北報導
金融研訓院參考入學考試的概念，推出金融人員基礎學科測驗，簡稱金融基測，未來求職者想要應徵公股銀行的職位，就可以基測作為參考成績，不用一家家招聘時都還要考一次筆試。研訓院宣布，明年開始金融基測就會上路，預計明年2月22日就會首次開考，以因應金融業上半年人力需求最旺盛的時期。
金融研訓院院長黃崇哲表示，研訓院受託辦理銀行業新進人員甄試多年，目前金融業、公股銀行個別委託研訓院辦理，每到徵才旺季大約是每年的第一季、第二季，考生得一家、一家報考，加上銀行近年面臨退休潮與環境變遷，研訓院思考要如何讓銀行、考生都可以都可以省時、省力與省錢。
所以研訓院推出類似入學考試、就是先測驗各個想要想要加入銀行體系的學生、新鮮人等的金融業基本能力，以及特殊專業像是洗錢或者法遵者需要的能力等。而各金融業所開出的條件裡，